### Understanding SQL Agents

In [ ]:
import os

from dotenv import load_dotenv
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.utilities import SQLDatabase

In [ ]:
load_dotenv(override=True)

azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_deployment = os.getenv("AZURE_DEPLOYMENT_NAME")
azure_api_version = os.getenv("AZURE_API_VERSION")

if not azure_endpoint or not azure_api_key or not azure_deployment or not azure_api_version:
    raise ValueError("Azure OpenAI environment variables are not set.")

temperature = 0.7
max_tokens = 1500

llm = AzureChatOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=azure_api_key,
    azure_deployment=azure_deployment,
    api_version=azure_api_version,
    temperature=temperature,
    max_tokens=max_tokens,
    model="gpt-4o"
)

In [ ]:
db_path = "sqlite:///./chinook.db"
db = SQLDatabase.from_uri(db_path)

print(db.dialect)
print(db.get_usable_table_names())

In [ ]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(
    llm=llm,
    db=db,
)

In [ ]:
question = "How many employees are there in the database?"

response = chain.invoke({
    "question": question
})

print("Question:", question)
print("Response:", response)

In [ ]:
question = "which country's customers have spent the most?"

chain = create_sql_query_chain(
    llm=llm,
    db=db,
)

response = chain.invoke({
    "question": question
})

print("Question:", question)
print("Response:", response)

In [ ]:
question = "How many employees are there in the database?"

response = chain.invoke({
    "question": question
})

print("Question:", question)
print("Response:", response)

In [ ]:
import re

# Extract SQL query from the response

sql_match = re.search(r'```sql\s*(.*?)\s*```', response, re.DOTALL)

if sql_match:
    response = sql_match.group(1).strip()
else:
    # Fallback: try to extract after "SQLQuery:" if no code block found
    if "SQLQuery:" in response:
        response = response.split("SQLQuery:")[1].split("SQLResult:")[0].strip()
        
db.run(response)

In [ ]:
chain.get_prompts()[0].pretty_print()

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool
from langchain_core.tools import tool
import re


@tool
def parse(query_string: str) -> str:
    """Parse the SQL query string and return a human-readable explanation."""
    
    # First, try to extract SQL query from code block with SQLQuery: prefix
    sql_match = re.search(r'SQLQuery:\s*```sql\s*(.*?)\s*```', query_string, re.DOTALL)
    
    if sql_match:
        response = sql_match.group(1).strip()
    else:
        # Try to extract SQL query from code block without prefix
        sql_match = re.search(r'```sql\s*(.*?)\s*```', query_string, re.DOTALL)
        
        if sql_match:
            response = sql_match.group(1).strip()
        else:
            # Fallback: try to extract after "SQLQuery:" if no code block found
            if "SQLQuery:" in query_string:
                response = query_string.split("SQLQuery:")[1].split("SQLResult:")[0].strip()
                # Remove any remaining backticks
                response = response.strip('`').strip()
            else:
                response = query_string.strip()
    
    return response

In [ ]:
parse("""
      ```sql SELECT COUNT("EmployeeId") AS EmployeeCount
        FROM Employee
        ```
      """)

In [ ]:
parse("""
      '--QUERY---': 
      ```sql
      SELECT COUNT("EmployeeId") AS EmployeeCount
        FROM Employee
        ```
      """)

In [ ]:
execute_query = QuerySQLDatabaseTool(db=db)
write_query = create_sql_query_chain(
    llm=llm,
    db=db,
)
chain = write_query | parse | execute_query
question = "How many employees are there in the database?"
response = chain.invoke({
    "question": question
})

print("Question:", question)
print("Response:", response)

In [ ]:
question = "which country's customers have spent the most? top 1 only"

execute_query = QuerySQLDatabaseTool(db=db)
write_query = create_sql_query_chain(
    llm=llm,
    db=db,
)
chain = write_query | parse | execute_query

response = chain.invoke({
    "question": question
})

print("Question:", question)
print("Response:", response)

In [ ]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [ ]:
answer_prompt = PromptTemplate.from_template(
    """
        Given the following user question, corresponding SQL Query, and SQL Result, answer the user question in a concise manner.
        
        User Question: {question}
        SQL Query: {query}
        SQL Result: {result}
        Answer: 
    """
)

chain = RunnablePassthrough \
    .assign(query=write_query) \
    .assign(result=itemgetter("query") | parse | execute_query) | \
    answer_prompt | llm | StrOutputParser()

In [ ]:
question = "How many employees are there in the database?"

response = chain.invoke({
    "question": question
})

print("Question:", question)
print("Response:", response)

In [ ]:
question = "which country's customers have spent the most? Give me the details also of the amount. I want the top 1 only."

response = chain.invoke({
    "question": question
})

print("Question:", question)
print("Response:", response)

In [ ]:
import ast
import re


def query_as_list(database, query):
    """
    Execute a SQL query and return the results as a list of dictionaries.
    """
    result = database.run(query)
    result = [el for sub in ast.literal_eval(result) for el in sub if el]
    result = [re.sub(r"\b\d+\b", "", string).strip() for string in result]

    return list(set(result))

artists = query_as_list(db, "SELECT Name FROM Artist")
albums = query_as_list(db, "SELECT Title FROM Album")

In [ ]:
artists[5]

In [ ]:
albums[:5]

In [ ]:
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain_community.vectorstores import FAISS

In [ ]:
azure_embedding_deployment = os.getenv(
    "AZURE_EMBEDDING_DEPLOYMENT_NAME", "text-embedding-3-large")

vector_database = FAISS.from_texts(
    texts=artists + albums,
    embedding=AzureOpenAIEmbeddings(azure_endpoint=azure_endpoint,
                                    model="text-embedding-3-large",
                                    api_key=azure_api_key,
                                    deployment=azure_embedding_deployment,
                                    )
)

retriever = vector_database.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2}
)

description = """
    use to lookup values to filter on.
    input is an approximate spelling of the valid and proper nouns.
    Use the noun most similar to the input.
    If the input is not a valid noun, return an empty string.
"""

retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="retriever",
    description=description,
)

In [ ]:
response = retriever_tool.invoke("Alis Chains")

print("Input: Alis Chains")
print("Response:", response)

In [ ]:
response = retriever_tool.invoke("Do we have any artists by named Alis Chains")

print("Input: Alis Chains")
print("Response:", response)